# Annotating the data
For the annotations of the sample I use the quantative content analysis (Lamnek 2005). Here three categories will be formed:
1. non-answer: The category encompasses every response where no reaction to the question occurs. Example: ""
2. evasive answer: This category is defined as reacting to the question in not or just partly answering the question. Example: "Sehr geehrter Herr W., haben Sie vielen Dank für Ihre Anfrage. Ich beteilige mich nicht länger am Portal abgeordnetenwatch.de. Um Ihre Frage dennoch zu beantworten, bitte ich um Mitteilung Ihrer E-Mail-Adresse an antje.tillmann@bundestag.de. Mit freundlichen Grüßen Antje Tillmann MdB"
3. answer: Every response which contains the answer to the questions in annotated in this category. Expample: "Sehr geehrter Herr Schellerich,die gesamte Fraktion DIE LINKE im Deutschen Bundestag wird dem ESM-Vertrag nicht zustimmen. Ich habe dies in meiner Rede vom 29.März im Bundestag auch versucht zu begründen. Mit freundlichen Grüßen Dr. Gysi"

The drawn sample will be mannualy annotated. Next the sample will be used to categorise the rest of the answers automatically.

In [215]:
# load libraries for data manipulation
import pandas as pd
import re
import regex
import numpy as np

# ML: Train/test splits, cross validation,
# gridsearch
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV,
)

# load libraries for tokenization
import nltk
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from nltk.corpus import stopwords
#nltk.download("stopwords")
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import preprocessing

# load libraries for text cleaning
import spacy
from spacy.lang.de.examples import sentences
# python -m spacy download de_core_news_sm
import ufal.udpipe
from gensim.models import KeyedVectors, Phrases
from gensim.models.phrases import Phraser
from ufal.udpipe import Model, Pipeline
import conllu

# Supervised text classification
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import joblib
#import eli5

## Preprocessing

In [216]:
# load data
sample_df = pd.read_csv("./data/stratified_sample.csv", sep=";")

In [217]:
# remove NaN for tokenizer to work
sample_df = sample_df.dropna(subset=["answer"])

Normalisierung von Umlauten

In [218]:
def remove_umlauts(text):    
    umlauts = {
        "ae" : "ä",
        "oe" : "ö",
        "ue" : "ü",
        "ss" : "ß"
    }

    for repl, original in umlauts.items():
        text = text.replace(original, repl)
    
    return text

In [219]:
sample_df["clean_answers"] = sample_df["answer"].apply(remove_umlauts)

The next step comprises the preprocessing of the data. All answers will be converted to lowercase, punctuation and other noise will be removed. Lowercasing each word has the advantage that there no two different writing styles of a word. I.e. "die" and "Die" are now recognized as the same word.

Lemmatisierung

In [220]:
nlp = spacy.load("de_core_news_sm")

def text_lemmatization(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc if not token.is_punct]
    return " ".join(lemmas)

In [221]:
def text_preprocessing(text):
    # remove links, punctuation, special letters
    text = re.sub(r"[^a-zA-Z]|\bhttps?://\S*|&\w+;|[\.,]", " ", text)
    
    # replace single characters
    text = re.sub(r" [a-zA-Z] ", " ", text)
    
    # remove additional whitespaces
    text = re.sub(r"\s+", " ", text)
    
    # lemmatize texts
    text = text_lemmatization(text)

    # lower text
    text = text.lower()
    
    # tokenization of words
    text = text.split()
    
    # remove stopwords
    german_stopwords = set(stopwords.words("german"))
    text = [w for w in text if w not in german_stopwords]
    
    # return joined text
    return " ".join(text)

In [222]:
sample_df["clean_answers"] = sample_df["clean_answers"].apply(text_preprocessing)

In [223]:
sample_df["clean_answers"]

0       geehrt herr vieler dank nachricht versorgung s...
1       geehrt herr vieler dank frage frage wertvoll f...
2       geehrt herr anpassung besoldung regelung infla...
3       geehrt frau vieler dank nachricht frage kranke...
4       geehrt herr blick moria schmerzen tatsache obd...
                              ...                        
2349    geehrt herr herzlich dank fuer frage februar b...
2350    geehrt herr vieler dank fuer frage bitte verst...
2352    geehrt herr burger bitte verstaendni herr lind...
2353    afd stehen fuer verkleinerung parlament vorsch...
2354    geehrt herr roth offensichtlich manipulation z...
Name: clean_answers, Length: 1867, dtype: object

In [224]:
# remove stopwords with refined list and after lemmatization to catch 
def remove_individual_stopwords(text):
    individual_stopwords = ["fuer", "gr", "nnen", "soll", "mehr", "weit", "freundlich", "herr", "frage", "frau", "dank", "ber", "gut", "vieler", "werden", "jahr", "viele", "ehre", "geehrt", "herzlich", "frage", "jahr", "geben"]
    text = text.split()
    text = [w for w in text if w not in individual_stopwords]
    
    return " ".join(text)

sample_df["clean_answers"] = sample_df["clean_answers"].apply(remove_individual_stopwords)

In [225]:
sample_df["clean_answers"]

0       nachricht versorgung sauber trinkwasser wesent...
1       wertvoll fachlich zust ndigkeit parlamentarisc...
2       anpassung besoldung regelung inflationspr mie ...
3       nachricht krankenversicherungsbeitr gen fallen...
4       blick moria schmerzen tatsache obdachlos gewor...
                              ...                        
2349    februar bundeskanzler olaf scholz praesident s...
2350    bitte verstaendni dafu beruflich entscheidung ...
2352    burger bitte verstaendni lindner kapazitaet za...
2353    afd stehen verkleinerung parlament vorschlag v...
2354    roth offensichtlich manipulation zustande geko...
Name: clean_answers, Length: 1867, dtype: object

In [226]:
# show most frequent words
vectorizer = TfidfVectorizer(
    min_df=5,
    max_df=0.5
)
dtm = vectorizer.fit_transform(sample_df["clean_answers"])

frequencies = dtm.sum(axis=0).tolist()[0]

freq_df = pd.DataFrame(
    dict(frequencies=frequencies,
         index=vectorizer.get_feature_names_out()
    )
)

freq_df.sort_values("frequencies", ascending=False).head(n=10)

,frequencies,index
3690,34.532199,sollen
2767,31.233002,mensch
1034,30.417955,deutschland
874,29.573698,bundestag
79,27.049973,afd
1032,26.982804,deutsch
3726,26.100131,spd
872,24.961109,bundesregierung
1940,24.418855,glich
1641,23.814459,finden


## Analyzing

In the next code chunk the sample data will be split into a training and test set. On the data of the training set the model will train and with the testing set the trained model will be tested. This step is necessary to avoid overfitting and ensure the quality of the results. This classifier functions as a baseline.

### Creating a pipeline
In the next step different pipelines are created to efficiently test and tune different vectorizers and classifiers. First the CountVectorizer(), the TfidfVectorizer(), the MultinomialNB() and the LogisticRegression() are used.

In [227]:
# split data into training and testing set with a testing set size of 20% of the data.
X_train, X_test, y_train, y_test = train_test_split(
    sample_df["clean_answers"],
    sample_df["answer_encoded"],
    test_size=0.2,
    random_state=42
)

pipes_and_grids = [
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", CountVectorizer()),
                ("classifier", MultinomialNB())
            ]
        ),
        "grid" : {
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
        }     
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", TfidfVectorizer()),
                ("classifier", MultinomialNB())
            ]
        ),
        "grid" : {
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
        }  
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", CountVectorizer()),
                ("classifier", LogisticRegression(solver="lbfgs"))
            ]
        ),
        "grid" : {
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
                    "classifier__C" : [0.01, 1, 100]
        }  
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", TfidfVectorizer()),
                ("classifier", LogisticRegression(solver="lbfgs"))
            ]
        ),
        "grid" : {
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
                    "classifier__C" : [0.01, 1, 100]
        }  
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", CountVectorizer()),
                ("classifier", RandomForestClassifier(n_estimators=100))
            ]
        ),
        "grid" : {
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
        }
    },
    {
        "pipeline" : Pipeline(
            steps=[
                ("vectorizer", TfidfVectorizer()),
                ("classifier", RandomForestClassifier(n_estimators=100))
            ]
        ),
        "grid" : {
                    "vectorizer__ngram_range" : [(1,1), (1,2), (1,3)],
                    "vectorizer__max_df" : [0.5, 1.0],
                    "vectorizer__min_df" : [1, 5],
                    "vectorizer__max_features" : [1000, 5000, 10000]
        }
    },
]

for pipe_and_grid in pipes_and_grids:
    search = GridSearchCV(
        estimator=pipe_and_grid["pipeline"], n_jobs=-1, param_grid=pipe_and_grid["grid"], scoring="accuracy", cv=10
    )

    search.fit(X_train, y_train)
    
    pred = search.predict(X_test)

    print(f"Vectorizer and classifier: {pipe_and_grid['pipeline']}")
    print(f"Best parameters: {search.best_params_}")

    rep = metrics.classification_report(y_test, pred)
    print(rep)

Vectorizer and classifier: Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('classifier', MultinomialNB())])
Best parameters: {'vectorizer__max_df': 0.5, 'vectorizer__min_df': 1, 'vectorizer__ngram_range': (1, 1)}
                precision    recall  f1-score   support

        answer       0.75      0.94      0.84       253
evasive answer       0.75      0.35      0.47       121

      accuracy                           0.75       374
     macro avg       0.75      0.65      0.66       374
  weighted avg       0.75      0.75      0.72       374

Vectorizer and classifier: Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', MultinomialNB())])
Best parameters: {'vectorizer__max_df': 0.5, 'vectorizer__min_df': 5, 'vectorizer__ngram_range': (1, 2)}
                precision    recall  f1-score   support

        answer       0.73      0.98      0.83       253
evasive answer       0.83      0.24      0.37       121

      accuracy           

- svm, knearest neighbors, gradient boost
- word embeddings
- question_text/_teaser

In [68]:
X_train, X_test, y_train, y_test = train_test_split(
    sample_df["clean_answers"],
    sample_df["answer_encoded"],
    test_size=0.2,
    random_state=42
)

vectorizer = TfidfVectorizer(
    ngram_range=(1,3)
)
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

svm = SVC(kernel='linear', C=1)
svm.fit(X_train_vect, y_train)
y_pred = svm.predict(X_test_vect)
print(metrics.classification_report(y_test, y_pred))


                precision    recall  f1-score   support

        answer       0.76      0.94      0.84       253
evasive answer       0.76      0.37      0.50       121

      accuracy                           0.76       374
     macro avg       0.76      0.66      0.67       374
  weighted avg       0.76      0.76      0.73       374

